# Simple "Chat with database case"

### Chat with data from GraphQL API with help from LangChain in a Gradio interface 

In [ ]:
# Import 
import os
from langchain import OpenAI
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory

import gradio as gr
import io
import sys


In [ ]:
OPENAI_API_KEY_HERE = 'openai-apikey'

In [ ]:


# Define the function to run the agent
def run_agent(Question):
    # Redirect stdout to capture the verbose logs for debugging
    old_stdout = sys.stdout
    new_stdout = io.StringIO()
    sys.stdout = new_stdout
    
    # Set the OpenAI API key as an environment variable
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY_HERE
    
    # Initialize the OpenAI model with a specific temperature setting
    llm = OpenAI(temperature=0)
    
    # Load GraphQL tool with a specific endpoint
    tools = load_tools(
        ["graphql"],
        graphql_endpoint="https://rickandmortyapi.com/graphql/",
    )
    
    # Initialize the agent with the given tools and model
    agent = initialize_agent(
        tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
    )
    
    # Define the GraphQL query to fetch character data from Rick and Morty API
    query_morty = """
    query {
        characters{
            results {
                name
                status
                species
                type
                gender
            }
        }
    }
    """
    
    # Run the agent with the user's question and the GraphQL query
    result = agent.run(Question + query_morty)
    
    # Capture the verbose logs 
    verbose_logs = new_stdout.getvalue()
    
    # Restore the original stdout stream
    sys.stdout = old_stdout
    
    # Filter the verbose logs to only include specific lines 
    filtered_logs = "\n".join([line for line in verbose_logs.split('\n') if "Thought:" in line or "Action Input:" in line or "Final Answer:" in line])
    
    # Combine the result and the filtered verbose logs for the final output
    combined_output = f"Result:\n{result}\n\n Verbose Logs:\n{filtered_logs}"

    return combined_output

# Create a Gradio interface for the agent function
iface = gr.Interface(
    fn=run_agent, 
    inputs=[
        gr.inputs.Textbox(lines=6, placeholder="Ask the database...")
    ], 
    title="Chat with Rick and Morty data via GraphQL Database API",
    description="Find entities using the `ChatGPT` with verbose to understand what's going on under the hood.",
    allow_flagging="never",
    examples=["What are the columns and the count of rows in the database?", "Can you list the names in the name column in the database?"],
    outputs="text"
)

# Launch the Gradio interface (set 'share' to True for a public URL)
iface.launch(share=False)
